In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00a 0:00:01


In [3]:
from datasets import load_dataset

ds = load_dataset("toughdata/quora-question-answer-dataset", split="train")

Generating train split:   0%|          | 0/56402 [00:00<?, ? examples/s]

In [4]:
ds = ds.train_test_split(test_size=0.2)

In [5]:
ds["train"][0]

{'question': "Is there a way to permanently delete files on a computer where even the FBI can't find the info?",
 'answer': 'TL;DR: While it might be theoretically possible to multiply overwrite the bytes of individual files to prevent recovery, in practical terms it’s extremely likely you’ll miss something, so you need to wipe the entire drive. For most people it will be sufficient to just use a tool like nwipe to overwrite the entire drive with gibberish several times. If you want paranoid levels of certainty, then you need to physically destroy the storage medium inside the drive.\n A friend recently asked me this about some drives with medical data that they needed to ensure were unrecoverable, for patient privacy reasons and legal due diligence. I gave them my advice, then double-checked it with a friend who does computer forensics, and my friend confirmed it.\n In a nutshell, as is almost always the case in technology questions, “it depends”… on how much time, energy and technica

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# def formatting_prompts_func(example):
#     output_texts = {}
#     for i in range(len(example['question'])):
#         text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
#         output_texts.update(text)
#     return output_texts

# processed_ds = ds.map(formatting_prompts_func, batched=True)

prefix = "Question: "

 
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["question"]]
    inputs = [doc + " Answer: " + doc2 for doc,doc2 in zip(inputs,examples["answer"])]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    return model_inputs

tokenized_ds = ds.map(preprocess_function, batched=True,remove_columns=ds["train"].column_names, )

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/45121 [00:00<?, ? examples/s]

Map:   0%|          | 0/11281 [00:00<?, ? examples/s]

In [7]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [8]:
lm_dataset = tokenized_ds.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/45121 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/11281 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

2024-07-30 06:44:54.157953: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 06:44:54.158089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 06:44:54.292216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("gpt2")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
training_args = TrainingArguments(
    output_dir="gpt2-qa",
    eval_strategy="epoch",
    learning_rate=2e-5,
    save_total_limit=3,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
#     predict_with_generate=True,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
#     compute_metrics = compute_metrics
)

In [12]:

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,3.235100,3.081926
2,3.068400,2.981990
3,3.013300,2.958890


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=6306, training_loss=3.1610127263624177, metrics={'train_runtime': 3089.8551, 'train_samples_per_second': 32.649, 'train_steps_per_second': 2.041, 'total_flos': 6589850370048000.0, 'train_loss': 3.1610127263624177, 'epoch': 3.0})

In [13]:
import math

eval_results = trainer.evaluate()
perplexity = math.exp(eval_results['eval_loss'])
eval_results['perplexity'] = perplexity

In [14]:
print(eval_results)

{'eval_loss': 2.9588897228240967, 'eval_runtime': 85.2137, 'eval_samples_per_second': 98.024, 'eval_steps_per_second': 6.138, 'epoch': 3.0, 'perplexity': 19.27655754797413}
